# Ingest Games Detail API to Bronze
This notebook loads the games detail data from the League of Legends API. Cf. detail on the API by Riot Games on: https://developer.riotgames.com/

For each game that we have in both the historical and recurring Summoner Games folders, we load games detail. Games detail contain all possible info about a given game: players that were in, champions played, amount of damage dealed and received, ...

This notebook requires the list of games for which we want to get data as input. We extract that list of games with the NB_Summoner_Games_API_to_Bronze notebook.

_Note: the games detail API only works for recent games. I'm not sure if it's a fixed amount of games or a given point in time but for my summoner, I could not load data before April 2023 (writing this as of June 2024)._

In [7]:
%run NB_Functions

### 1. General configuration

In [2]:
# Import libraries
from trident_token_library_wrapper import PyTridentTokenLibrary as tl
import requests as r
import json
import os
from datetime import datetime
from notebookutils import mssparkutils

In [3]:
# Load secrets from Azure Key Vault
key_vault_name = 'testasa-akv-dev-001'
key_name = 'lol-api-key'
access_token = mssparkutils.credentials.getToken("keyvault")
api_key = tl.get_secret_with_token(f"https://{key_vault_name}.vault.azure.net/", key_name, access_token)

# Define API endpoint and parameters
base_url = 'https://europe.api.riotgames.com/lol/'

### 2. Load games detail data

In [33]:
# Define the date from which we want to load games detail.
LH_ID = mssparkutils.lakehouse.get("LKH_Bronze").id
WS_ID = mssparkutils.lakehouse.get("LKH_Bronze").workspaceId
try:
    files = get_dir_content(f'abfss://{WS_ID}@onelake.dfs.fabric.microsoft.com/{LH_ID}/Files/raw_data/games_detail')

    if files: # if we already loaded some games data, only get games detail for games starting from this point on
        startDate = get_most_recent_date(files)

    else: # if the games detail folder is empty
        startDate = "2024-06-14" # date at which recent games were loaded. Sadly, most of historical games are too old for the games detail API to work...

except Exception as e: # if the games detail folder does not yet exist
    startDate = "2024-06-14"
    print("Error", e )

print(startDate)

In [34]:
# Get the list of games we want to get detail from

# Filter summoner_games files
base_path = "Files/raw_data/summoner_games/recurring/"
games_files = get_dir_content(base_path, extension=".json")

print("Reading summoner games into a DataFrame")

df_games = spark.read.json(games_files)
print("Found", df_games.count(), "games")

df_filtered_games = df_games.select("game").filter(df_games.date >= startDate) # load only games from which we did not already get detail
print("Filtered", df_filtered_games.count(), "games")

df_unique_games = df_filtered_games.dropDuplicates(["game"]) # as multiple summoners may have played in the same game, dropping duplicates is needed
print("Kept", df_unique_games.count(), "unique games")

print("Finished reading summoner games into the Dataframe, loading", df_unique_games.count(), "new games.")

# Make a list out of the games to iterate on games
games_list = df_unique_games.collect()

In [ ]:
# Make API calls to get games detail for each game

current_date = datetime.now().strftime("%Y/%m/%d")

print("Starting API calls to get games detail")
for game_row in games_list:
    print(f"Getting data for game: {game_row[0]}")
    game = game_row[0]

    # Make API call to fetch game detail
    params = {'api_key': api_key}
    response = r.get(f"{base_url}match/v5/matches/{game}", params=params)

    if response.status_code == 200:
        outputFilePath = f"//lakehouse/default/Files/raw_data/games_detail/{current_date}/{game}.json"
 
        if not os.path.exists(os.path.dirname(outputFilePath)):    
            print("creating folder: " + os.path.dirname(outputFilePath))    
            os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
 
        with open(outputFilePath, 'wb') as f:
            f.write(response.content)

    else:
        print(f"Failed to fetch game detail for game {game}, https status code: {response.status_code}")

    time.sleep(1) # see API limits: https://developer.riotgames.com/docs/portal